# 문제 6

[Kaggle 형] train_prob.csv로 문제 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [2]:
df_train = pd.read_csv('train_prob.csv', index_col=['id'])
df_test = pd.read_csv('test_prob.csv', index_col=['id'])
df_test_ans = pd.read_csv('test_prob.csv', index_col=['id'])

In [4]:
df_train = df_train.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna()
)

df_test = df_test.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna()
)

In [20]:
df_train['loading'] = df_train['loading'].fillna(df_train['loading'].mean())
df_test['loading'] = df_test['loading'].fillna(df_train['loading'].mean())

In [5]:
df_train['product_code'].value_counts()

C    5765
E    5343
B    5250
A    5100
Name: product_code, dtype: int64

In [6]:
df_test['product_code'].value_counts()

D    5112
Name: product_code, dtype: int64

In [7]:
# 방법 1: product_code 별로 IterativeImputer를 통한 결측처리를 하고 바로 적용합니다.

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

imp = IterativeImputer(
    estimator = LinearRegression(),
    random_state=123
)

X_iter = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']
df_train[X_iter] = df_train.groupby('product_code')[X_iter].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x[X_iter]), index=x.index, columns=x.columns)
)
df_test[X_iter] = df_test.groupby('product_code')[X_iter].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x[X_iter]), index=x.index, columns=x.columns)
)

In [8]:
X_mean = ['measurement_{}'.format(i) for i in range(10, 17)]
df_train[X_mean] = df_train.groupby('product_code')[X_mean].transform(lambda x: x.fillna(x.mean()))
df_test[X_mean] = df_test.groupby('product_code')[X_mean].transform(lambda x: x.fillna(x.mean()))

In [ ]:
# loading: log 변환
# SFS ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17', 'na_1']
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis,  Classifier, Transformer
# PCA(n_components=7)
# RandomForestClassifier {'max_depth': 7, 'min_samples_split': 512, 'n_estimators': 15} 0.574 튜닝

In [14]:
from sklearn.model_selection import GroupKFold, cross_validate

X_all =df_test.columns.tolist()
gcv = GroupKFold(n_splits=4)
for train_idx, valid_idx in gcv.split(df_train[X_all], df_train['failure'], groups=df_train['product_code']):
    print(
        df_train.iloc[train_idx]['product_code'].unique(),
        df_train.iloc[valid_idx]['product_code'].unique()
    )

['A' 'B' 'E'] ['C']
['A' 'B' 'C'] ['E']
['A' 'C' 'E'] ['B']
['B' 'C' 'E'] ['A']


In [21]:
from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(solver='lbfgs')
X_lr = ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17', 'na_1']
result = cross_validate(
    clf_lr, df_train[X_lr], df_train['failure'], 
    cv=gcv, groups=df_train['product_code'], 
    scoring='roc_auc', return_train_score=True
)
result

{'fit_time': array([0.08676457, 0.08970904, 0.04887724, 0.05969834]),
 'score_time': array([0.        , 0.        , 0.00101471, 0.        ]),
 'test_score': array([0.5827344 , 0.58852714, 0.57957189, 0.58180685]),
 'train_score': array([0.58973664, 0.59171369, 0.58047486, 0.57853698])}